In [13]:
import random
import pandas as pd

In [14]:
room_excel = pd.read_excel("Dataset/Gedung TULT.xlsx", usecols=['Nama Ruangan'])
room_list = room_excel['Nama Ruangan'].tolist()
room_list

['TULT-0601',
 'TULT-0602',
 'TULT-0603',
 'TULT-0701',
 'TULT-0702',
 'TULT-0703',
 'TULT-0706',
 'TULT-0707',
 'TULT-0709',
 'TULT-0710',
 'TULT-0711',
 'TULT-0714',
 'TULT-0715',
 'TULT-0716',
 'TULT-1507',
 'TULT-1509',
 'TULT-1510']

In [15]:
class_excel = pd.read_excel("Dataset/Kelas FIF.xlsx", sheet_name="Kelas FIF", usecols=['Kelas FIF'])
class_list = class_excel['Kelas FIF'].tolist()
class_list

['IF-47-01',
 'IF-47-02',
 'IF-47-03',
 'IF-47-04',
 'IF-47-05',
 'IF-47-06',
 'IF-47-07',
 'IF-47-08',
 'IF-47-09',
 'IF-47-10',
 'IF-47-11',
 'IF-47-12',
 'IF-47-INT',
 'IT-47-01',
 'IT-47-02',
 'IT-47-03',
 'IT-47-04',
 'SE-47-01',
 'SE-47-02',
 'SE-47-03',
 'SE-47-04',
 'DS-47-01',
 'DS-47-02',
 'DS-47-03']

In [16]:
subject_excel = pd.read_excel("Dataset/Mata Kuliah FIF.xlsx", sheet_name="FIF Semester 1", usecols=['Kode'])
subject_list = subject_excel['Kode'].tolist()
subject_list

['UKJXB2',
 'UAJXA2',
 'CII1E3',
 'CII1A3',
 'CII1C2',
 'CII1B3',
 'CII1D3 ',
 'CTJ1A2',
 'CRI1A2',
 'CRI1B3',
 'CSI1A2']

In [17]:
# Read the dataset and filter by status
dosen_dataset = pd.read_excel("Dataset/data dosen.xlsx", sheet_name="all dosen", usecols=['Kode', 'Status'])

In [18]:
rooms = room_list
students_groups = class_list
subjects = subject_list
permanent_lectures = dosen_dataset.loc[dosen_dataset['Status'] == 1, 'Kode'].tolist()
non_permanent_lectures = dosen_dataset.loc[dosen_dataset['Status'] == 0, 'Kode'].tolist()
lecturers = permanent_lectures + non_permanent_lectures
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
timeslots = ['timeslot1', 'timeslot2', 'timeslot3', 'timeslot4', 'timeslot5', 'timeslot6', 'timeslot7', 'timeslot8', 'timeslot9', 'timeslot10', 'timeslot11', 'timeslot12']

In [19]:
# Hyperparameters
num_ants = 10
num_iterations = 100
decay = 0.5
alpha = 1
beta = 2

In [20]:
# Hyperparameters
num_ants = 10
num_iterations = 100
decay = 0.5
alpha = 1
beta = 2

In [21]:
# Timeslots to minimize usage
undesirable_timeslots = ['timeslot1', 'timeslot5', 'timeslot6', 'timeslot11']

# Initial pheromone levels
pheromone = [[[1 for _ in timeslots] for _ in rooms] for _ in days]
pheromone_lecturer = {lecturer: 1 for lecturer in lecturers}

# Define the fitness function
def fitness(schedule):
    penalty = 0
    room_timeslot_usage = {}
    lecturer_course_group = {}
    timeslot_count = {room: {day: 0 for day in days} for room in rooms}
    group_timeslot_count = {group: {day: 0 for day in days} for group in students_groups}

    room_conflicts = 0
    lecturer_conflicts = 0
    undesirable_timeslot_penalty = 0
    empty_timeslot_penalty = 0
    specific_rules_penalty = 0

    for group in schedule:
        for (course, day, room, timeslot, lecturer) in schedule[group]:
            # Check room usage
            if (day, room, timeslot) in room_timeslot_usage:
                room_conflicts += 1
            else:
                room_timeslot_usage[(day, room, timeslot)] = group
                timeslot_count[room][day] += 1
                group_timeslot_count[group][day] += 1

            # Check lecturer constraints
            if (lecturer, course, group) in lecturer_course_group:
                lecturer_conflicts += 1
            else:
                lecturer_course_group[(lecturer, course, group)] = True

            # Additional penalty for undesirable timeslots
            if timeslot in undesirable_timeslots:
                undesirable_timeslot_penalty += 0.5

            # Specific rule: No permanent lecturers on timeslot4, timeslot5, timeslot6 on Thursdays
            if day == 'Thursday' and timeslot in ['timeslot4', 'timeslot5', 'timeslot6'] and lecturer in permanent_lectures:
                specific_rules_penalty += 1

            # Specific rule: No courses in timeslot5 and timeslot6 on Friday
            if day == 'Friday' and timeslot in ['timeslot5', 'timeslot6']:
                specific_rules_penalty += 1

            # Specific rule: Rooms TULT-1507, TULT-1509, TULT-1510 only for IF-47-INT
            if room in ['TULT-1507', 'TULT-1509', 'TULT-1510'] and group != 'IF-47-INT':
                specific_rules_penalty += 1

            # One course per timeslot per day per group
            if group_timeslot_count[group][day] > 1:
                specific_rules_penalty += 1

    # Penalty for empty timeslots and unbalanced room usage
    for room in rooms:
        for day in days:
            empty_timeslot_penalty += (len(timeslots) - timeslot_count[room][day]) * 0.1  # Less penalty for fewer empty slots

    total_penalty = (room_conflicts + lecturer_conflicts + undesirable_timeslot_penalty + empty_timeslot_penalty + specific_rules_penalty)

    return total_penalty



In [22]:
# Create the initial population
def create_schedule():
    schedule = {group: [] for group in students_groups}
    for group in students_groups:
        for course in subjects:
            try:
                course_length = int(course[-1])  # Get the length of the course from the last digit
            except ValueError:
                continue  # Skip if the course code does not end with a digit

            while True:
                day = random.choice(days)
                room = random.choice(rooms)
                start_timeslot_index = random.randint(0, len(timeslots) - course_length)
                timeslot_range = timeslots[start_timeslot_index:start_timeslot_index + course_length]
                lecturer = random.choice(lecturers)

                if any([(day, room, timeslot) in [(d, r, t) for c, d, r, t, l in schedule[group]] for timeslot in timeslot_range]):
                    continue  # Skip if any timeslot in the range is already taken

                if (room in ['TULT-1507', 'TULT-1509', 'TULT-1510'] and group == 'IF-47-INT'):
                    continue  # Skip if room restriction is violated

                if (day == 'Thursday' and any(timeslot in ['timeslot4', 'timeslot5', 'timeslot6'] for timeslot in timeslot_range) and lecturer in permanent_lectures):
                    continue  # Skip if permanent lecturer rule is violated

                if (day == 'Friday' and any(timeslot in ['timeslot5', 'timeslot6'] for timeslot in timeslot_range)):
                    continue  # Skip if Friday timeslot rule is violated

                for timeslot in timeslot_range:
                    schedule[group].append((course, day, room, timeslot, lecturer))
                break
    return schedule




In [23]:
# Update pheromones
def update_pheromones(pheromone, pheromone_lecturer, all_schedules):
    for day in range(len(days)):
        for room in range(len(rooms)):
            for timeslot in range(len(timeslots)):
                pheromone[day][room][timeslot] *= decay
                for schedule in all_schedules:
                    if is_course_scheduled(schedule, day, room, timeslot):
                        pheromone[day][room][timeslot] += 1 / (1 + fitness(schedule))
    
    for lecturer in lecturers:
        pheromone_lecturer[lecturer] *= decay
        for schedule in all_schedules:
            if is_lecturer_scheduled(schedule, lecturer):
                pheromone_lecturer[lecturer] += 1 / (1 + fitness(schedule))

    return pheromone, pheromone_lecturer

# Check if a course is scheduled
def is_course_scheduled(schedule, day, room, timeslot):
    for group in schedule:
        for (course, d, r, t, lecturer) in schedule[group]:
            if d == days[day] and r == rooms[room] and t == timeslots[timeslot]:
                return True
    return False

# Check if a lecturer is scheduled
def is_lecturer_scheduled(schedule, lecturer):
    for group in schedule:
        for (course, day, room, timeslot, l) in schedule[group]:
            if l == lecturer:
                return True
    return False

# Main ACO loop
def aco():
    best_schedule = None
    best_fitness = float('inf')
    for iteration in range(num_iterations):
        all_schedules = []
        for ant in range(num_ants):
            schedule = create_schedule()
            all_schedules.append(schedule)
            fit = fitness(schedule)
            if fit < best_fitness:
                best_fitness = fit
                best_schedule = schedule
        update_pheromones(pheromone, pheromone_lecturer, all_schedules)
    return best_schedule, best_fitness



In [24]:
# Display the best schedule in a timetable form categorized by room
def print_timetable_by_room(schedule):
    timetable = {room: {day: {timeslot: None for timeslot in timeslots} for day in days} for room in rooms}
    for group in schedule:
        for (course, day, room, timeslot, lecturer) in schedule[group]:
            timetable[room][day][timeslot] = (course, group, lecturer)

    for room in rooms:
        print(f"Timetable for {room}:")
        for day in days:
            print(f"  {day}:")
            for timeslot in timeslots:
                entry = timetable[room][day][timeslot]
                if entry:
                    course, group, lecturer = entry
                    print(f"    {timeslot}: {course} for {group} with {lecturer}")
                else:
                    print(f"    {timeslot}: Free")
        print()

best_schedule, best_fitness = aco()
print_timetable_by_room(best_schedule)
print("Best Fitness:", best_fitness)

Timetable for TULT-0601:
  Monday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: CRI1B3 for IT-47-01 with DRI
    timeslot4: CRI1B3 for IT-47-01 with DRI
    timeslot5: CII1A3 for SE-47-04 with DYA
    timeslot6: CII1A3 for SE-47-04 with DYA
    timeslot7: CII1A3 for SE-47-04 with DYA
    timeslot8: CII1C2 for SE-47-02 with AZN
    timeslot9: CII1E3 for IF-47-10 with FMH
    timeslot10: Free
    timeslot11: CSI1A2 for IF-47-08 with BRV
    timeslot12: CSI1A2 for IF-47-08 with BRV
  Tuesday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: UAJXA2 for IF-47-11 with ADE
    timeslot6: UAJXA2 for IF-47-11 with ADE
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free
    timeslot11: UKJXB2 for IF-47-07 with ALH
    timeslot12: UKJXB2 for IF-47-07 with ALH
  Wednesday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: CRI1B3 for SE-47-04 with IPL
    timeslot4: CRI1B3 for SE-47-04 with IPL
    timeslot5: 